In [7]:
import os

base_path = "data/real-vs-fake"
subsets = ["train", "test", "valid"]
classes = ["real", "fake"]

total_counts = {"train": {"real": 0, "fake": 0},
                "test": {"real": 0, "fake": 0},
                "valid": {"real": 0, "fake": 0}}

for subset in subsets:
    for cls in classes:
        path = os.path.join(base_path, subset, cls)
        if os.path.exists(path):
            count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg'))])
            total_counts[subset][cls] = count
        else:
            print(f" Pasta não encontrada: {path}")


for subset in subsets:
    real = total_counts[subset]["real"]
    fake = total_counts[subset]["fake"]
    total = real + fake
    print(f"\n {subset.upper()}:")
    print(f" - Imagens reais: {real}")
    print(f" - Imagens falsas: {fake}")
    print(f" - Total: {total}")

total_real = sum(total_counts[s]["real"] for s in subsets)
total_fake = sum(total_counts[s]["fake"] for s in subsets)
print(f"\n Total geral de imagens reais: {total_real}")
print(f" Total geral de imagens falsas: {total_fake}")
print(f" Total geral de imagens: {total_real + total_fake}")



 TRAIN:
 - Imagens reais: 50000
 - Imagens falsas: 50000
 - Total: 100000

 TEST:
 - Imagens reais: 10000
 - Imagens falsas: 10000
 - Total: 20000

 VALID:
 - Imagens reais: 10000
 - Imagens falsas: 10000
 - Total: 20000

 Total geral de imagens reais: 70000
 Total geral de imagens falsas: 70000
 Total geral de imagens: 140000
